# Python для сбора и обработки данных

Автор: Валентин Бирюков

# Базы данных в юпитере - sqllite

In [2]:
!pip install --user --upgrade ipython-sql -q

In [3]:
%load_ext sql
%sql sqlite://

Пара замечаний:
 * Мы используем веб-интерфейс для python, поэтому для запуска SQL запросов необходимо применить ряд вещей:
     * Необходимо подключить расширение SQL через так называемые magic command. [Более подробно здесь](https://ipython.readthedocs.io/en/stable/interactive/magics.html)
     * Для работы с SQL надо вызвать либо %sql для однострочной команды, либо %%sql - для многострочной
     * При использовании выражения SELECT результатом вывода является таблица, но при этом в notebook выводится уже внутренне представление языка python (объект класса sql.run.resultset, подробности далее), что приводит к ряду несоответствий (например, None вместо NULL)

In [4]:
%%sql drop table if exists product;
create table product(
       pname        varchar primary key, -- имя продукта
       price        money,               -- цена продукта
       category     varchar,             -- категория
       manufacturer varchar NOT NULL     -- производитель
);
insert into product values('Пряник', 19.99, 'Еда', 'Тульский пряник');
insert into product values('Тетрадь', 29.99, 'Канцелярия', 'Эксмо');
insert into product values('Печенье', 149.99, 'Еда', 'WagonWiels');
insert into product values('Планшет', 203.99, 'Техника', 'Zua');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Посмотрим на полученную таблицу.

In [5]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Пряник,19.99,Еда,Тульский пряник
Тетрадь,29.99,Канцелярия,Эксмо
Печенье,149.99,Еда,WagonWiels
Планшет,203.99,Техника,Zua


In [6]:
product_result=%sql select * from product

 * sqlite://
Done.


In [7]:
print(type(product_result))
print(product_result)

<class 'sql.run.ResultSet'>
+---------+--------+------------+-----------------+
|  pname  | price  |  category  |   manufacturer  |
+---------+--------+------------+-----------------+
|  Пряник | 19.99  |    Еда     | Тульский пряник |
| Тетрадь | 29.99  | Канцелярия |      Эксмо      |
| Печенье | 149.99 |    Еда     |    WagonWiels   |
| Планшет | 203.99 |  Техника   |       Zua       |
+---------+--------+------------+-----------------+


Немного SQL терминологии
--------------------------
* _имя_ таблицы - product.
* Каждая строка таблицы называется _строкой_ или _кортеж_. 
* Заметьте, что все кортежи имеют _поля_ или _атрибуты_.
* Количество строк называет _мощностью_, в то время как количество атрибутов _арностью_ 

Соглашения схемы
-----------------
* Схема таблицы продукта может быть описана следующим образом:

> product(<u>pname</u>, price, category, manufacturer)

Подчеркнутое множество атрибутов формирует  _ключ_.

* В этом случае ключ - pname. Если имя продукта было бы только уникальным для указнного производителя мы бы написали:

> product(<u>pname</u>, price, category, <u>manufacturer</u>)


Описание таблиц. Допущения 
----------------
* Кортеж = запись (На текушем уровне понимания, на самом деле это не так)
  * Ограничение: все атрибуты - атомарного типа (это тоже не совсем корректное утверждение, так как существует довольно сложные типы)
  * Существует множество атомарных типов для SQL, например, по postgresQL можно посмотреть [здесь](http://www.postgresql.org/docs/9.4/static/datatype.html)
  .


* Таблица = мультимножество кортежей 
  * Мультимножество похоже на список
  * ... Но по определению оно не упорядочено: 
    * нет first(), нет next(), нет last().

# Простые запросы
* Рассмотрим основы SQL на примере
* В интернете очень много SQL туториалов, сейчас мы всего лишь вспомним базовые вещи

> SELECT <Множество атрибутов><br>
> FROM <список таблиц и условие на соединение><br>
> WHERE <список условий>

Это простейший SELECT-FROM-WHERE (SFW) блок. Давайте посмотрим на примерах!

In [8]:
%%sql
SELECT * 
  FROM Product 
 WHERE category = 'Канцелярия' 
   AND manufacturer = 'Эксмо'


 * sqlite://
Done.


pname,price,category,manufacturer
Тетрадь,29.99,Канцелярия,Эксмо


Посмотрим на примеры *проекции*, то есть получим только несколько атрибутов запроса.

In [9]:
%%sql 
SELECT Pname, 
       Price, 
       Manufacturer
  FROM Product;

 * sqlite://
Done.


pname,price,manufacturer
Пряник,19.99,Тульский пряник
Тетрадь,29.99,Эксмо
Печенье,149.99,WagonWiels
Планшет,203.99,Zua


На выходе запроса к таблице - снова таблица 
---------------------------------

In [11]:
%%sql
SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT distinct p0.category
     FROM Product p0
     WHERE p0.price < 20.00) cp, -- this is a nested query!
    Product p
WHERE 
    p.category = cp.category and p.price > 20.00

 * sqlite://
Done.


manufacturer,pname,price
WagonWiels,Печенье,149.99


Небольшие детали
--------------

* Некоторые элементы регистро-независимые:
  * Одно и то же: SELECT  Select  select
  * Одно и то же: Product   product
  * Разные: ‘Seattle’  ‘seattle’ (Здесь это строковая константа)
    * Строго говоря, в большинстве случаев это зависит от настроек СУБД
  

* Константы (одинарные кавычки)
  * ‘abc’  - да
  * “abc” - нет


In [12]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Пряник,19.99,Еда,Тульский пряник
Тетрадь,29.99,Канцелярия,Эксмо
Печенье,149.99,Еда,WagonWiels
Планшет,203.99,Техника,Zua


LIKE
====

Опертоор LIKE нужен для поиска строк:
    
> SELECT *
> FROM Products
> WHERE pname like '%gizmo%'

Like - "сильно урезанный" regexp
* % - сколько угодно символов
* \_ ровно один символ
* оператор LIKE - регистрозависимый

In [14]:
%%sql
SELECT *  FROM product
where category LIKE '%да%'

 * sqlite://
Done.


pname,price,category,manufacturer
Пряник,19.99,Еда,Тульский пряник
Печенье,149.99,Еда,WagonWiels


Убрать дубли
---------------------
* Дубли не всегда хорошо, и иногда их стоит убирать
  * Помните, что таблицы - _мультимножества_!

In [15]:
%sql SELECT category from product;

 * sqlite://
Done.


category
Еда
Канцелярия
Еда
Техника


In [16]:
%%sql 
-- чтобы убрать дубли используйте слово DISTINCT
SELECT DISTINCT category from product;

 * sqlite://
Done.


category
Еда
Канцелярия
Техника


Сортировка результатов
---------------------
* Так как Таблица - это мультимножество, то порядок вывода строк не гарантирован. Иногда необходимо выводить строки в определенном порядке


In [17]:
%%sql
-- сортировка результатов
-- сортировка по умолчанию - ascending
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price > 50
ORDER BY  price, pname

 * sqlite://
Done.


pname,price,manufacturer
Печенье,149.99,WagonWiels
Планшет,203.99,Zua


In [18]:
%%sql
-- сортировка результатов
-- тип сортировки каждого компонента определяется индивидуально
SELECT   price, manufacturer
FROM     Product
ORDER BY   manufacturer ASC, price DESC

 * sqlite://
Done.


price,manufacturer
149.99,WagonWiels
203.99,Zua
19.99,Тульский пряник
29.99,Эксмо


Можно делать сортировку по порядковому номеру, но довольно часто это считается bad practice 

In [19]:
%%sql
SELECT   price, manufacturer
FROM     Product
ORDER BY   1 ASC, 2 DESC

 * sqlite://
Done.


price,manufacturer
19.99,Тульский пряник
29.99,Эксмо
149.99,WagonWiels
203.99,Zua


Работа с несколькими таблицами
------


* Рассмотрим таблицу компаний.
> company(<u>cname</u>, stockprice, country)
 

In [21]:
%%sql
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('Тульский пряник', 25.0, 'Россия');
insert into company values ('Эксмо', 65.0, 'Россия');
insert into company values ('Zua', 15.0, 'Китай');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Внешние ключи
-----------------------
* Допустим мы ходим добавить таблицу продуктов

> Product(pname, price, category, manufacturer)

* Может возникнуть следующая ситуация: есть компания, продающая какие-то продукты, но при этом она отсутствует в таблице компаний.
* Чтобы избежать это, воспользуемя _внешними ключами_ 

Назвнание компании в product _ссылается_ на поле cnma из таблицы company:

> foreign key (manufacturer) references company(cname)

  * Замечание: cname <u>должно быть</u> ключом в  company! 
  

In [22]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING by default off in sqlite
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));

insert into product values('Пряник', 19.99, 'Еда', 'Тульский пряник');
insert into product values('Тетрадь', 29.99, 'Канцелярия', 'Эксмо');
insert into product values('Печенье', 149.99, 'Еда', 'Тульский пряник');
insert into product values('Планшет', 203.99, 'Техника', 'Zua');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Внешние ключи являются _ограничениями_ таблицы 
> Что случится при вставке компании, которой не в таблице company?


In [23]:
try:
    %sql insert into product values('MultiTouch', 203.99, 'Household', 'Google');
except Exception as e:
    print (e)
    print ("Rejected!")

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "insert into product values('MultiTouch', 203.99, 'Household' , 'Google' );"] (Background on this error at: http://sqlalche.me/e/gkpj)
Rejected!


In [24]:
%%sql
-- the update is rejected!
select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Пряник,19.99,Еда,Тульский пряник
Тетрадь,29.99,Канцелярия,Эксмо
Печенье,149.99,Еда,Тульский пряник
Планшет,203.99,Техника,Zua


Внешние ключи и удаление
=============

* Что произойдет, если мы удалим компанию ? Несколько вариантов:
  * Запретить удаление (default)
  * Удалить все продукты (добавьте "`on delete cascade`")
  * Замена на  NULL или DEFAULT
  

In [26]:
try:
    %sql delete from company where cname = 'Zua';
except Exception as e:
    print (e)
    print ("Disallowed!")

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "delete from company where cname = 'Zua' ;"] (Background on this error at: http://sqlalche.me/e/gkpj)
Disallowed!


**Второй вариант: удалить все продукты, принадлежащие компании, которую мы удаляем**

Необходимо изменить опеределение в create table:
> foreign key (manufacturer) references company(cname) on delete cascade

Теперь, когда строка компании удалена, все продукты с указанной компанией, будут также удалены.

Соединения
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, stockPrice, country)

Мы хотим ответить на вопрос

> Найти все продукты меньшие 200$, произведенные в России
> Возвратить их имена и цену. 



In [29]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Россия' and price <= 200;

 * sqlite://
Done.


pname,price
Пряник,19.99
Тетрадь,29.99
Печенье,149.99


Другой вариант написания запроса.

In [31]:
%%sql -- Часть 1: выбрать все компании из России
SELECT distinct cname -- нужна ли нам уникальность?
from company where country='Россия';

 * sqlite://
(sqlite3.OperationalError) near "Часть": syntax error [SQL: "Часть 1: выбрать все компании из России\nSELECT distinct cname -- нужна ли нам уникальность?\nfrom company where country='Россия';"] (Background on this error at: http://sqlalche.me/e/e3q8)


In [32]:
%%sql -- Part 2: Продукты до 200
select distinct pname, price, manufacturer
from product
where price <= 200;

 * sqlite://
(sqlite3.OperationalError) near "Part": syntax error [SQL: 'Part 2: Продукты до 200\nselect distinct pname, price, manufacturer\nfrom product\nwhere price <= 200;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [33]:
%%sql -- Объединение как cross join
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Россия') JapaneseProducts;

 * sqlite://
(sqlite3.OperationalError) near "Объединение": syntax error [SQL: "Объединение как cross join\nSELECT * \nFROM \n  (SELECT DISTINCT pname, price, manufacturer\n   FROM product\n   WHERE price <= 200) CheapProducts,\n  (SELECT DISTINCT cname\n   FROM company\n   WHERE country='Россия') JapaneseProducts;"] (Background on this error at: http://sqlalche.me/e/e3q8)


In [34]:
%%sql
-- Фильтруем cross join
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Россия') JapaneseProducts
WHERE cname = manufacturer;

 * sqlite://
Done.


pname,price
Печенье,149.99
Пряник,19.99
Тетрадь,29.99


# Использование конструкции JOIN

Есть другой вариант записи - через конструкцию JOIN. Кроме этого она позволяет делать также внешние запросы.



In [36]:
%%sql
SELECT pname, 
       price
  FROM product p 
  JOIN company c 
    ON p.manufacturer = c.cname
 WHERE c.country='Россия' 
   AND p.price <= 200;

 * sqlite://
Done.


pname,price
Пряник,19.99
Тетрадь,29.99
Печенье,149.99


Примечания
--------
* Есть множество логических вариантов написать один и тот же запрос
    * Этот факт будет использоваться для оптимизации